# Create a Dynamic RTree backend

By default cogeo-mosaic backends were meant to handle writing and reading mosaicjson either from a file or from a database.

While this is fine for most use cases, some users could want something more `dynamic`. In this Notebook we will show how to create a Dynamic mosaic backend based on RTree (https://rtree.readthedocs.io/en/latest/tutorial.html#using-rtree-as-a-cheapo-spatial-database).



## Requirements

To be able to run this notebook you'll need the following requirements:
- cogeo-mosaic
- rtree
- shapely
- tqdm

In [ ]:
# Uncomment this line if you need to install the dependencies
# !pip install cogeo-mosaic rtree shapely tqdm

In [ ]:
import requests
import pickle

from tqdm.notebook import tqdm

from rtree import index

## 1. Create the rtree Index

Ref: https://azure.microsoft.com/en-us/services/open-datasets/catalog/naip/?tab=data-access#AzureNotebooks

Azure is hosting a RTree index (tile_index) and a binary file with all the Naip geometry (tiles.p)

binary: https://naipeuwest.blob.core.windows.net/naip-index/rtree/tiles.p
Rtree: https://naipeuwest.blob.core.windows.net/naip-index/rtree/tile_index.dat and https://naipeuwest.blob.core.windows.net/naip-index/rtree/tile_index.idx

Sadly the Rtree contains only the Indexes, which then has to be used to retrieve the path and geometry in `tiles.p`

For this Demo we need to store the information directly in the Rtree object 


In [ ]:
# Download geometry file
url = "https://naipeuwest.blob.core.windows.net/naip-index/rtree/tiles.p"
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open("tiles.p", "wb") as f:
        for chunk in r.iter_content(chunk_size=8192): 
            f.write(chunk)

# Load tile index and create rtree index
with open("tiles.p", "rb") as f:
    tile_index = pickle.load(f)

In [ ]:
# Create the Cheapo Rtree database
# Make sure naip.dat and naip.idx do not exists
naip_index = index.Rtree('naip')
for idx, (f, geom) in tqdm(tile_index.items(), total=len(tile_index)):
    naip_index.insert(idx, geom.bounds, obj=f"https://naipeuwest.blob.core.windows.net/naip/{f}")
naip_index.close()

## 2. Create backend

In [ ]:
from typing import Dict, Tuple, Type, Optional, List, Callable

import attr
from morecantile import TileMatrixSet
from rio_tiler.constants import WEB_MERCATOR_TMS
from rio_tiler.io import BaseReader
from rio_tiler.io import STACReader

from cogeo_mosaic.backends.base import BaseBackend
from cogeo_mosaic.backends.stac import _fetch, default_stac_accessor
from cogeo_mosaic.mosaic import MosaicJSON


@attr.s
class DynamicRtreeBackend(BaseBackend):
    
    path: str = attr.ib()
    reader: Type[BaseReader] = attr.ib(default=STACReader)
    reader_options: Dict = attr.ib(factory=dict)
    backend_options: Dict = attr.ib(factory=dict)

    query: Dict = attr.ib(factory=dict)

    # default values for bounds and zoom
    bounds: Tuple[float, float, float, float] = attr.ib(
        init=False,
        default=(-180, -90, 180, 90)
    )
    minzoom: int = attr.ib(default=0)
    maxzoom: int = attr.ib(default=30)

    # Because we are not using mosaicjson we are not limited to the WebMercator TMS
    tms: TileMatrixSet = attr.ib(default=WEB_MERCATOR_TMS)

    asset_filter: Callable = attr.ib(default=lambda x: x)
        
    # The reader is read-only, we can't pass mosaic_def to the init method
    mosaic_def: MosaicJSON = attr.ib(init=False)

    index = attr.ib(init=False)

    _backend_name = "DynamicSTAC"

    def __attrs_post_init__(self):
        """Post Init."""
        # Construct a FAKE mosaicJSON
        # mosaic_def has to be defined. As we do for the DynamoDB and SQLite backend
        # we set `tiles` to an empty list.
        self.mosaic_def = MosaicJSON(
            mosaicjson="0.0.2",
            name="it's fake but it's ok",
            minzoom=self.minzoom,
            maxzoom=self.maxzoom,
            tiles=[]
        )
        self.index = index.Index(self.path)
        self.bounds = tuple(self.index.bounds)

    def close(self):
        """Close SQLite connection."""
        self.index.close()

    def __exit__(self, exc_type, exc_value, traceback):
        """Support using with Context Managers."""
        self.close()        
        
    def write(self, overwrite: bool = True):
        """This method is not used but is required by the abstract class."""
        pass

    def update(self):
        """We overwrite the default method."""
        pass

    def _read(self) -> MosaicJSON:
        """This method is not used but is required by the abstract class."""
        pass

    def assets_for_tile(self, x: int, y: int, z: int) -> List[str]:
        """Retrieve assets for tile."""
        bbox = self.tms.bounds(x, y, z)
        return self.get_assets(bbox, **self.backend_options)

    def assets_for_point(self, lng: float, lat: float) -> List[str]:
        """Retrieve assets for point."""
        EPSILON = 1e-14
        bbox = (lng - EPSILON, lat - EPSILON, lng + EPSILON, lat + EPSILON)
        return self.get_assets(bbox, **self.backend_options)

    def get_assets(self, bbox, asset_filter: Callable = lambda x: x) -> List[str]:
        """Find assets."""
        assets = [n.object for n in self.index.intersection(bbox, objects=True)]
        return self.asset_filter(assets)

    @property
    def _quadkeys(self) -> List[str]:
        return []


In [ ]:
# Get assets for a Tile requests
with DynamicRtreeBackend("naip") as mosaic:
    print(mosaic.assets_for_tile(4684, 6278, 14))

The naip dataset has couple overlapping years, to create an optimized mosaic we need to filter the assets.

Here is an example of filter function which takes the latest data and highest resolution first.

In [ ]:
import pathlib

def latest_naip_asset(assets: List[str]) -> List[str]:
    
    def get_info(asset) -> Dict:
        parts = pathlib.Path(asset).parts
        capture_date = parts[-1].split("_")[-1].rstrip(".tif")
        resolution = int(parts[-3].split("_")[1].rstrip("cm"))
        fname_parts = parts[-1].split("_")
        quadrangle = f"{fname_parts[1]}_{fname_parts[2]}_{fname_parts[3]}"
    
        return {
            "path": asset,
            "capture_date": capture_date,
            "quadrangle": quadrangle,
            "resolution": resolution
        }

    asset_info = [get_info(f) for f in assets]
    
    # Sort by resolution and by dates
    asset_info = sorted(
        asset_info, key=lambda item: (item["capture_date"], -item["resolution"]),
        reverse=True
    )

    quad = []
    out_dataset = []
    for d in asset_info:
        q = d["quadrangle"]
        if q not in quad:
            out_dataset.append(d["path"])
            quad.append(q)

    return out_dataset


In [ ]:
with DynamicRtreeBackend("naip", asset_filter=latest_naip_asset) as mosaic:
    print(mosaic.assets_for_tile(4684, 6278, 14))